<a href="https://colab.research.google.com/github/aemiralfath/BPML-Dicoding/blob/master/nlp_2_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
! pip install --upgrade --force-reinstall --no-deps kaggle
! mkdir /root/.kaggle/
! cp "/content/drive/My Drive/Machine Learning/kaggle.json" /root/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=7b12356be4306621df0a4a5045c5a56b90984bca0f5fce6d814b2abad4060d6e
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6


In [3]:
!kaggle datasets download -d marklvl/sentiment-labelled-sentences-data-set

  0% 0.00/326k [00:00<?, ?B/s]
100% 326k/326k [00:00<00:00, 77.4MB/s]


In [4]:
import zipfile

zipref = zipfile.ZipFile('sentiment-labelled-sentences-data-set.zip', 'r')
zipref.extractall()
zipref.close()

In [59]:
import pandas as pd

df = pd.read_csv('sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')

In [60]:
df.tail()

,sentence,label
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0
999,"Then, as if I hadn't wasted enough of my life ...",0


In [61]:
from sklearn.model_selection import train_test_split

sentences = df['sentence'].values
y = df['label'].values

x_train, x_test, y_train, y_test = train_test_split(sentences, y, test_size=0.2)

In [62]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(x_train)

sequence_train = tokenizer.texts_to_sequences(x_train)
sequence_test = tokenizer.texts_to_sequences(x_test)

padded_train = pad_sequences(sequence_train, padding='post', maxlen=30, truncating='post')
padded_test = pad_sequences(sequence_test, padding='post', maxlen=30, truncating='post')

In [63]:
print(padded_train)
print(padded_test)

[[ 18 780  31 ...   0   0   0]
 [ 53 164 288 ...   0   0   0]
 [  4  49  89 ...   0   0   0]
 ...
 [  2  18  31 ...   0   0   0]
 [  9   8  22 ...   0   0   0]
 [ 37   4  32 ...   0   0   0]]
[[ 150  373   53 ...    0    0    0]
 [  67    0    0 ...    0    0    0]
 [   1   24    6 ...    0    0    0]
 ...
 [ 622 1643  438 ...    0    0    0]
 [   4  104  486 ...    0    0    0]
 [  72    8  122 ...    0    0    0]]


In [64]:
print(len(padded_train[0]))
print(len(padded_test[0]))

30
30


In [69]:
import tensorflow as tf

model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(10000, 64, input_length=30),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                            #  tf.keras.layers.GlobalAveragePooling1D(),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 30, 64)            640000    
_________________________________________________________________
dropout_9 (Dropout)          (None, 30, 64)            0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 30, 128)           66048     
_________________________________________________________________
dropout_10 (Dropout)         (None, 30, 128)           0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 128)             

In [70]:
num_epochs = 30
history = model.fit(
    padded_train,
    y_train,
    epochs=num_epochs,
    validation_data=(padded_test, y_test)
)

Epoch 1/30
25/25 [==============================] - 2s 65ms/step - loss: 0.6947 - accuracy: 0.4888 - val_loss: 0.6924 - val_accuracy: 0.5500
Epoch 2/30
25/25 [==============================] - 0s 19ms/step - loss: 0.6919 - accuracy: 0.5088 - val_loss: 0.6885 - val_accuracy: 0.6150
Epoch 3/30
25/25 [==============================] - 0s 19ms/step - loss: 0.6695 - accuracy: 0.6037 - val_loss: 0.5969 - val_accuracy: 0.6650
Epoch 4/30
25/25 [==============================] - 0s 19ms/step - loss: 0.4632 - accuracy: 0.8150 - val_loss: 0.4313 - val_accuracy: 0.8350
Epoch 5/30
25/25 [==============================] - 0s 19ms/step - loss: 0.2357 - accuracy: 0.9212 - val_loss: 0.4110 - val_accuracy: 0.8300
Epoch 6/30
25/25 [==============================] - 0s 19ms/step - loss: 0.1600 - accuracy: 0.9525 - val_loss: 0.5723 - val_accuracy: 0.7900
Epoch 7/30
25/25 [==============================] - 0s 19ms/step - loss: 0.1143 - accuracy: 0.9663 - val_loss: 0.4239 - val_accuracy: 0.8550
Epoch 8/30
25